In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2022-11-16 14:40:22.188226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 14:40:22.333577: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 14:40:22.812693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-16 14:40:22.812769: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
import tensorflow_datasets as tfds

/home/mikheil/.conda/envs/ssl-repo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sys.path.append(str(Path("../../ssl").resolve()))

In [5]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.mean_teacher.mean_teacher import MeanTeacherTrainer
from src.trainers.mean_teacher.mean_teacher_config import MeanTeacherTrainerConfig
from src.data_loaders.pi_model.pi_model_config import PiModelDataLoaderConfig
from src.data_loaders.pi_model.pi_model import PiModelDataLoader

# Description

In this notebook, a model will be trained on the CIFAR-10 dataset using only 20% of the labelled dataset. Unlabelled data will be used in training.

## Set up Experiment

In [6]:
class TrainerConfig(MeanTeacherTrainerConfig):
    num_epochs = 200
    loss_ramp_up_epochs = 150
    unsup_loss_weight = 3.0
    alpha = 0.99

train_config = TrainerConfig()

In [7]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [8]:
class DataLoaderConfig(PiModelDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 50000 # dataset size    

    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config = DataLoaderConfig()

## Get Datasets

In [9]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [10]:
# get train dataset by subsampling 20% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.20, random_state = 42
)

# cast types
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.int64)
y_train_labelled = y_train_labelled.astype(np.int64)
y_test_full = y_test_full.astype(np.int64)

In [11]:
x_train = np.concatenate((x_train_unlabelled, x_train_labelled), axis = 0)
y_train = np.concatenate((y_train_unlabelled, y_train_labelled), axis = 0)

In [12]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-16 14:40:24.906506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 14:40:24.914564: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 14:40:24.915270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 14:40:24.916626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [13]:
# create train dataset
train_data = PiModelDataLoader(train_data, data_loader_config)(training = True)

In [14]:
# create test dataset
val_data = PiModelDataLoader(val_data, data_loader_config)(training = False)

In [15]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")

Train dataset size: 782
Validation dataset size: 157


# Train Model

In [16]:
model_student = PiModel(model_config)()
model_teacher = PiModel(model_config)()

In [17]:
trainer = MeanTeacherTrainer(
    model_student,
    model_teacher,
    train_data,
    train_config,
    val_dataset = val_data)

In [18]:
trainer.train()

2022-11-16 14:40:27.950413: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : 2.15. Validation loss is : 1.94. Validation acc. is : 24.59.
Training loss at epoch 1 is : 1.93. Validation loss is : 1.78. Validation acc. is : 34.40.
Training loss at epoch 2 is : 1.83. Validation loss is : 1.67. Validation acc. is : 37.34.
Training loss at epoch 3 is : 1.76. Validation loss is : 1.63. Validation acc. is : 40.69.
Training loss at epoch 4 is : 1.72. Validation loss is : 1.60. Validation acc. is : 40.22.
Training loss at epoch 5 is : 1.66. Validation loss is : 1.50. Validation acc. is : 44.64.
Training loss at epoch 6 is : 1.60. Validation loss is : 1.42. Validation acc. is : 48.50.
Training loss at epoch 7 is : 1.56. Validation loss is : 1.38. Validation acc. is : 50.42.
Training loss at epoch 8 is : 1.51. Validation loss is : 1.36. Validation acc. is : 50.60.
Training loss at epoch 9 is : 1.49. Validation loss is : 1.29. Validation acc. is : 53.92.
Training loss at epoch 10 is : 1.44. Validation loss is : 1.23. Validation acc. is : 55.28